# 整理Netflix电影演员评分数据

## 分析目的

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netfix电视剧及电影数据。数据集包含两个数据表: titles.csv 和credits.csv 。  

tit1es.csv 包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB(一个国外的在线评分网站)评分，等等。  

credits.csv 包含超过7万名出现在Netfix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型(导演/演员)等。  
titles.csv 每列的含义如下:  

id: 影视作品ID。  

title: 影视作品标题。  

show type: 作品类型，电视节目或电影,  

description: 简短描述。  

release year: 发布年份。  

age certification: 适龄认证  

runtime: 每集电视剧或电影的长度。  

genres: 流派类型列表。  

production countries: 出品国家列表。  

seasons: 如果是电视剧，则是季数。  

imdb id: lMDB的ID。  

imdb score: lMDB的评分。  

imdb votes: IMDB的投票数  

tmdb popularity: TMDB的流行度。  

tmdb score: TMDB的评分。  

credits.csv: 每列的含义如下:  

person lD: 演职员ID。  

id: 参与的影视作品ID。  

name: 姓名。  

character name: 角色姓名。  

role: 演职员类型，演员或导演。  


## 一、读取数据

In [4]:
import pandas as pd
import ast
original_credits=pd.read_csv(r'D:\my_project\credits.csv')
original_titles=pd.read_csv(r'D:\my_project\titles.csv')


## 二、评估和清洗数据

从结构和内容两个维度(整齐度和干净度）对数据进行评估。结构：每列一个变量，每行一个观察值，每个单元格一个值；内容：缺失、重复、无效、错误  
为了区分开经过清理的数据和原始的数据，我们创建新的变量cleaned_titles，cleaned_credits。我们之后的清理步骤都将被运用在cleaned_titles和  
cleaned_credits上。

### 1.评估数据结构 ——整齐度

In [5]:
original_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


从展现出来的数据来看，数据符合每列一个变量，每行一个观察值，每个单元格一个值不存在结构性问题。

In [6]:
original_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,NaN,100,"['romance', 'drama']",['NG'],NaN,tt13857480,6.8,45.0,1.466,NaN
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021,NaN,134,['drama'],[],NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,['comedy'],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021,PG-13,37,[],['US'],NaN,NaN,NaN,NaN,1.296,10.000


从展现出来的数据来看，数据不符合每列一个变量，每行一个观察值，每个单元格一个值不存在结构性问题。  

genres、	production_countries每行存在多个值应该进行拆分

创建副本，进行清洗

In [7]:
cleaned_credits=original_credits.copy()
cleaned_titles=original_titles.copy()
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,NaN,100,"['romance', 'drama']",['NG'],NaN,tt13857480,6.8,45.0,1.466,NaN
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021,NaN,134,['drama'],[],NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,['comedy'],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021,PG-13,37,[],['US'],NaN,NaN,NaN,NaN,1.296,10.000


cleaned_titles['genres'] =cleaned_titles ['genres'].apply(lambda x:ast.literal_eval(x))
cleaned_titles

In [8]:
cleaned_titles['genres'] =cleaned_titles ['genres'].apply(lambda x:ast.literal_eval(x))
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,[documentation],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"[drama, crime]",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"[drama, action, thriller, european]",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"[fantasy, action, comedy]",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"[war, action]","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5845,tm1014599,Fine Wine,MOVIE,A beautiful love story that can happen between...,2021,NaN,100,"[romance, drama]",['NG'],NaN,tt13857480,6.8,45.0,1.466,NaN
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021,NaN,134,[drama],[],NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,[comedy],['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021,PG-13,37,[],['US'],NaN,NaN,NaN,NaN,1.296,10.000


genres是字符串，无法直接用value_counts统计各个值出现的次数，需要把他转换为列表

In [9]:
cleaned_titles = cleaned_titles.explode('genres')
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,['CO'],NaN,tt14585902,3.8,68.0,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021,PG-13,37,NaN,['US'],NaN,NaN,NaN,NaN,1.296,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,family,[],1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,animation,[],1.0,tt13711094,7.8,18.0,2.289,10.000


In [10]:
import ast
cleaned_titles['production_countries']=cleaned_titles['production_countries'].apply(lambda x:ast.literal_eval(x))
cleaned_titles['production_countries']

0       [US]
1       [US]
1       [US]
2       [US]
2       [US]
        ... 
5847    [CO]
5848    [US]
5849      []
5849      []
5849      []
Name: production_countries, Length: 15147, dtype: object

In [11]:
cleaned_titles=cleaned_titles.explode('production_countries')
cleaned_titles.sample(50)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
4866,ts328194,My Name,SHOW,"Following her father's murder, a revenge-drive...",2021,TV-MA,50,crime,KR,1.0,tt12940504,7.8,19737.0,86.286,8.300
5284,ts269286,Once Upon a Time... Happily Never After,SHOW,A tragically separated couple must find each o...,2022,TV-MA,30,scifi,ES,1.0,tt13560186,5.4,496.0,73.753,7.500
3111,tm943790,Summerland,MOVIE,A woman during the Second World War opens her ...,2020,PG,102,drama,GB,NaN,tt6841122,7.0,6827.0,9.770,7.600
1366,ts54100,World of Winx,SHOW,The Winx travel all over the world searching f...,2016,TV-Y7,24,european,US,2.0,tt5874596,6.6,727.0,6.824,8.200
1917,ts79660,Creeped Out,SHOW,"A masked figure known as ""The Curious"" collect...",2017,TV-PG,26,horror,CA,2.0,tt7573024,6.6,1890.0,24.556,7.300
2778,tm138667,Taramani,MOVIE,An orthodox youngster and a free-spirited lady...,2017,NaN,150,drama,IN,NaN,tt4328934,7.5,1639.0,2.616,6.600
5564,tm1063013,The Father Who Moves Mountains,MOVIE,"Mircea, former Intelligence officer, finds out...",2021,PG-13,108,thriller,SE,NaN,tt8886670,5.5,3472.0,19.955,6.000
4950,tm314863,Vivo,MOVIE,A music-loving kinkajou named Vivo embarks on ...,2021,PG,103,drama,US,NaN,tt6338498,6.7,18238.0,101.830,7.500
15,tm89386,Hitler: A Career,MOVIE,A keen chronicle of the unlikely rise to power...,1977,PG,150,history,DE,NaN,tt0191182,7.5,2460.0,4.305,7.300
1335,tm233616,LEGO Marvel Super Heroes: Avengers Reassembled!,MOVIE,The Avengers are forced to “party” with Ultron...,2015,NaN,22,family,US,NaN,tt5371572,6.1,862.0,16.252,6.500


到这一步为止，数据符合每列一个变量，每行一个观察值，每个单元格一个值，已经不存在结构性问题

### 2.评估数据结构 ——干净度

注意缺失值，以及当前数据类型是否是正常的，不符合的要进行格式转换，检查错误值、缺失值、重复值。

In [12]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


character存在缺失值，person_id应该为字符串类型，而不是整数

In [13]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

title, description,age_certification,genres,production_countries,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score 均存在不同程度的缺失。
release_year需要进行数据转换，release_year是日期型数据。

### 数据类型转换

In [14]:
cleaned_credits['person_id']=cleaned_credits['person_id'].astype(str)
cleaned_credits.info()    #数据类型转换

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  object
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: object(5)
memory usage: 3.0+ MB


In [15]:
cleaned_titles['release_year']=pd.to_datetime(cleaned_titles['release_year'],format='%Y')
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021-01-01,PG-13,37,NaN,US,NaN,NaN,NaN,NaN,1.296,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


### 缺失值处理

In [16]:
cleaned_credits[cleaned_credits['character'].isnull()]

,person_id,id,name,character,role
36,3308,tm84618,Martin Scorsese,NaN,DIRECTOR
59,17727,tm154986,John Boorman,NaN,DIRECTOR
106,11475,tm127384,Terry Jones,NaN,DIRECTOR
107,11473,tm127384,Terry Gilliam,NaN,DIRECTOR
162,1063,tm120801,Robert Aldrich,NaN,DIRECTOR
...,...,...,...,...,...
77776,2363022,tm1097142,Mohamed El-Arkan,NaN,ACTOR
77777,1827884,tm1097142,Mohamed Bakir,NaN,DIRECTOR
77783,678884,tm1014599,Segun Arinze,NaN,ACTOR
77789,1962840,tm1014599,Seyi Babatope,NaN,DIRECTOR


本次分析目标是挖掘出各个流派中的高评分作品演员，角色的缺失不影响最终结果的核算，因此可以保留character的缺失值

In [17]:
cleaned_titles[cleaned_titles['genres'].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
212,tm255589,One Last Shot,MOVIE,"In this low-budget short film, two best buddie...",1998-01-01,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,1.890,5.2
619,tm341561,Like Twenty Impossibles,MOVIE,Occupied Palestine: A serene landscape now poc...,2003-01-01,NaN,16,NaN,PS,NaN,NaN,NaN,NaN,0.812,6.5
632,ts86241,Le Robe De Mariage Des Cieux,SHOW,It was with much difficulty that Ai Qing was a...,2004-01-01,TV-MA,63,NaN,NaN,1.0,NaN,NaN,NaN,0.600,NaN
636,tm404676,To and from New York,MOVIE,"While covering a story in New York City, a Sea...",2006-01-01,NaN,82,NaN,US,NaN,NaN,NaN,NaN,1.401,5.8
637,tm89054,Osuofia in London 2,MOVIE,Osuofia return to his Nigerian village with a ...,2004-01-01,NaN,72,NaN,XX,NaN,NaN,NaN,NaN,1.091,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,tm1040959,The Circle: The Afterparty,MOVIE,Stars of The Circle drop by to discuss Season ...,2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,1.882,10.0
5802,ts302434,Plastic Cup Boyz: Laughing My Mask Off!,SHOW,Comedy collective The Plastic Cup Boyz pour ou...,2021-01-01,NaN,33,NaN,NaN,1.0,NaN,NaN,NaN,0.683,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


可以观察到，genres为空，imdb_score也为空，可以验证一下，是否imdb_score，genres同时为空

In [18]:
cleaned_titles[(cleaned_titles['genres'].isnull()) & (cleaned_titles['imdb_score'].notnull())]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,2018-01-01,TV-MA,50,NaN,US,4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",2018-01-01,TV-PG,52,NaN,US,1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,2018-01-01,TV-G,24,NaN,US,1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,2018-01-01,TV-PG,68,NaN,NaN,1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,2020-01-01,NaN,49,NaN,NaN,NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,2019-01-01,NaN,27,NaN,NaN,NaN,tt10545994,7.7,NaN,NaN,NaN


结果证明，genres，imdb_score并不同时为空，但是由于genres，imdb_score有空值，genres的缺失，使得imdb_score的存在也没有意义，因此，需要删除genres的缺失值以及imdb_score的缺失值。

In [19]:
cleaned_titles = cleaned_titles.dropna(subset=['genres'])

In [20]:
cleaned_titles = cleaned_titles.dropna(subset=['imdb_score'])

In [21]:
cleaned_titles.isnull().sum()

id                          0
title                       0
type                        0
description                 5
release_year                0
age_certification        6464
runtime                     0
genres                      0
production_countries      300
seasons                 11016
imdb_id                     0
imdb_score                  0
imdb_votes                 29
tmdb_popularity           128
tmdb_score                455
dtype: int64

### 重复值处理

In [22]:
cleaned_credits.duplicated().sum()

0

In [23]:
cleaned_titles.duplicated().sum()

0

以上两个表均不存在重复值，因此不做处理

### 处理不一致

In [27]:
with pd.option_context('display.max_rows',None):
    print(cleaned_titles['production_countries'].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
KR          637
ES          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
NL          102
HK          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
AT           28
IS           28
LU           27
NZ           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

从以上输出结果来看，出品国家都用两位的国家代码来表示，除了里面存在一个的Lebanon值。

Lebanon的国家代码是LB，出现了39次，说明此处数据不一致。LB和Lebanon都在表示同一国家，需要进行统一。

把cleaned_titles里，production_countries的"LB"和"Lebanon"统一为LB，并检查替换后是否还存在"LB"：

In [30]:
cleaned_titles['production_countries']=cleaned_titles['production_countries'].replace({'Lebanon':'LB'})

In [32]:
with pd.option_context('display.max_rows',None):
    print(cleaned_titles['production_countries'].value_counts())

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
KR     637
ES     637
CA     608
DE     383
CN     295
MX     264
IT     224
BR     221
AU     217
TR     195
PH     192
AR     150
ID     149
BE     148
TW     133
NG     131
PL     126
ZA     103
HK     102
NL     102
CO      94
EG      93
DK      89
TH      87
SE      81
LB      71
NO      68
AE      52
IE      49
SG      47
XX      43
IL      42
RU      41
CL      35
CH      33
PS      32
BG      31
MY      30
AT      28
SA      28
IS      28
LU      27
NZ      27
PE      26
RO      25
QA      24
CZ      22
JO      19
FI      18
HU      18
UY      15
MA      15
PT      14
KW      10
KH      10
PK       9
PR       9
UA       8
MT       8
VN       8
LT       7
IR       7
CD       7
SU       7
TN       7
SN       6
AL       6
KE       6
GH       6
IQ       5
MU       5
CY       5
KN       4
GR       4
IO       4
SY       4
TZ       4
MC       4
GL       3
CM       3
AO       3
BS       3
HR       3
BD       3


# 三、整合数据

In [33]:
cleaned_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


In [34]:
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021-01-01,NaN,134,drama,NaN,NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


为了获得完整的分析结果，我们需要把两个表连接起来，以id键进行连接，因为两个表里都有id键盘，通过id键，可以匹配上对应的数据进行分析。

In [38]:
credits_titles=pd.merge(cleaned_credits,cleaned_titles,on='id',how='inner')
credits_titles

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276104,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
276105,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
276106,373198,tm1059008,Inés Prieto,Fanny,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
276107,378132,tm1059008,Isabel Gaona,Cacica,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300


In [46]:
credits_titles_Actor = credits_titles.query('role == "ACTOR"')

选择数据，只留ACTOR的数据

In [48]:
credits_titles_Actor.head(20)

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
5,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
6,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
7,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
8,48933,tm84618,Cybill Shepherd,Betsy,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
9,48933,tm84618,Cybill Shepherd,Betsy,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179


分组，聚合，按照genres，person_id进行分组

In [63]:
credits_titles_Actor_Mean=credits_titles_Actor.groupby(['genres','person_id'])['imdb_score'].mean().reset_index()
credits_titles_Actor_Mean

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


In [64]:
credits_titles_Actor_Mean_Max=credits_titles_Actor_Mean.groupby('genres')['imdb_score'].max().reset_index()
credits_titles_Actor_Mean_Max

,genres,imdb_score
0,action,9.3
1,animation,9.3
2,comedy,9.2
3,crime,9.5
4,documentation,9.1
5,drama,9.5
6,european,8.9
7,family,9.3
8,fantasy,9.3
9,history,9.1


In [58]:
Last_merge=pd.merge(credits_titles_Actor_Mean_Max,credits_titles_Actor_Mean,on=['genres','imdb_score'],how='inner')

In [59]:
Last_merge

,genres,imdb_score,person_id
0,action,9.3,12790
1,action,9.3,1303
2,action,9.3,21033
3,action,9.3,336830
4,action,9.3,86591
...,...,...,...
131,war,8.8,826547
132,western,8.9,22311
133,western,8.9,28166
134,western,8.9,28180


提取cleaned_credits中的person_id、name，去重

In [71]:
id_name=cleaned_credits[['person_id','name']]
id_name_Drop_Dup=id_name.drop_duplicates()
id_name_Drop_Dup

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
...,...,...
77796,736339,Adelaida Buscato
77797,399499,Luz Stella Luengas
77798,373198,Inés Prieto
77799,378132,Isabel Gaona


In [72]:
Last_merge_name=pd.merge(Last_merge,id_name_Drop_Dup,on='person_id',how='inner')

Last_merge_name

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
1,action,9.3,1303,Jessie Flower
2,action,9.3,21033,Zach Tyler
3,action,9.3,336830,André Sogliuzzo
4,action,9.3,86591,Cricket Leigh
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
132,western,8.9,22311,Koichi Yamadera
133,western,8.9,28166,Megumi Hayashibara
134,western,8.9,28180,Unsho Ishizuka


In [76]:
Last_merge_name.sort_values(by='imdb_score',ascending=False).reset_index()

,index,genres,imdb_score,person_id,name
0,28,drama,9.5,37752,Anna Gunn
1,128,thriller,9.5,37752,Anna Gunn
2,127,thriller,9.5,20552,Betsy Brandt
3,21,crime,9.5,85485,RJ Mitte
4,20,crime,9.5,37752,Anna Gunn
...,...,...,...,...,...
131,78,music,8.8,3015,Tom Berenger
132,77,music,8.8,3014,Marion Cotillard
133,76,music,8.8,3013,Dileep Rao
134,75,music,8.8,3012,Cillian Murphy


排序，重新设置索引